In [1]:
import numpy as np, matplotlib.pyplot as plt, scipy.signal as ss, scipy.fftpack as sf
from scipy.signal import hilbert

import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import utility_fx as util

In [2]:
%matplotlib qt5
%qtconsole --style native

In [3]:
from scipy.signal import butter, lfilter

def filter_data(data, low,high, sf=32000, order=2):
    # Determine Nyquist frequency
    nyq = sf/2
    # Set bands
    low = low/nyq
    high = high/nyq
    # Calculate coefficients
    b, a = butter(order, [low, high], btype='band')
    # Filter signal
    filtered_data = lfilter(b, a, data)

    return filtered_data

In [4]:
datafile = "E:\\Ratterdam\\R781\\Beltway_D3_190307\\"
expCode = "BRD3"

figpath = f"E:\\Ratterdam\\R781\\beltway_test_plots\\{expCode}\\"
beltwayAlleys = [16,17,3,1,5,7,8,10,11]

ratplot = Vis.BasicRateMaps()

alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
stimfiles = Parse.getStimFileName(datafile)
stimData = Parse.loadBeltwayData(datafile,stimfiles,expCode)


alleySwapTS = Parse.beltway_getLapTimes(datafile, stimData, expCode)

In [5]:
tt = 9
fname = datafile + f"CSC{tt}.ncs"

In [6]:
ncsType = np.dtype([
    ('ts', '<u8'),
    ('dwChannelNumber', '<u4'),
    ('fs', '<u4'),
    ('NumValidSamples', '<u4'),
    ('data', '<i2', (512, ))
])

In [7]:
curFile = open(fname)
curFile.seek(16*1024)
curFile = np.fromfile(curFile, dtype=ncsType)

In [8]:
dt = (1/32000)*1e6 # this is the timestep between each sample in packet. Only first sample is explicitly timestamped
                     # by neuralynx at time of collection so increment this dt value repeatedly to get ts of all other samples in packet
cscdata = curFile['data'].flatten()
cscts = []
for packet in curFile:
    init_ts = packet['ts']
    cscts.append(init_ts)
    interp_ts = init_ts + dt # initialize the, basically, counter. this init is for the second point if that makes sense
    for datapoint in packet['data'][1:]: # start with second bc first one is (the only one) timestamped
        cscts.append(interp_ts)
        interp_ts += dt # increment for next ts
        
cscts = np.asarray(cscts)

#### end data loading and preprocessing

In [142]:
alley=5 # real id
visitTS = alleyVisits[alley-1]
cscchunks = []
tschunks = []

for visit in visitTS:
    mask = (cscts>visit[0])&(cscts<visit[1])
    cscchunks.append(cscdata[mask])
    tschunks.append(cscts[mask])

In [69]:
specs = []
downsample_factor = 1
for data in cscchunks:
    f,t,s = ss.spectrogram(data[::downsample_factor],fs=32000/downsample_factor)
    s = 10*np.log10(s)
    specs.append(s)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log10
  """


In [70]:
fig, ax = plt.subplots(5,6,figsize=(10,10))
for i, s in enumerate(specs):
    fig.axes[i].imshow(s, origin='lower',aspect='auto',interpolation='None')
    fig.axes[i].set_ylim([4,12])
    fig.axes[i].set_title(txtVisits[7-1][i])

In [143]:
visits_theta = []
visits_gamma = []

for i,visit in enumerate(alleyVisits[alley-1]):
    theta = filter_data(cscchunks[i], 8,12)
    gamma = filter_data(cscchunks[i], 25, 45)
    visits_theta.append(theta)
    visits_gamma.append(gamma)

In [144]:
hilbert3 = lambda x: hilbert(x, fftpack.next_fast_len(len(x)))[:len(x)] # this truncates bc for certain array sizes (primes) underlying FFT is insanely slow.
phases = []
for i in range(len(cscchunks)):
    analytic = hilbert3(visits_gamma[i])
    phase = np.angle(analytic)
    phases.append(phase)

In [123]:
unit = Core.UnitData("TT6\\cl-maze1.4", datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
unit.loadData_raw(includeRewards=True)

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:157: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:157: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:163: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:163: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:321: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


In [20]:
# subplots for each visit with theta, gamma, and spike overlays
numrows = 5
fig, ax = plt.subplots(numrows,int(np.ceil(len(alleyVisits[alley-1])/numrows)), figsize=(10,10))

for i,visit in enumerate(alleyVisits[alley-1]):
    fig.axes[i].plot(tschunks[i], visits_theta[i], 'k', alpha=0.5)
    fig.axes[i].plot(tschunks[i], visits_gamma[i], 'g', alpha=0.5)
    fig.axes[i].vlines(unit.alleys[alley][i]['spikes'][:,0], -10000,10000, color='r')
    fig.axes[i].set_title(f"{i}, {txtVisits[alley-1][i]}")
    fig.axes[i].set_xticks([])

In [25]:
# subplots for each visit with phases of whatever you filtered above with spike overlays
numrows = 5
fig, ax = plt.subplots(numrows,int(np.ceil(len(alleyVisits[alley-1])/numrows)), figsize=(10,10))

for i,visit in enumerate(alleyVisits[alley-1]):
    fig.axes[i].plot(tschunks[i], phases[i], 'k', alpha=0.5)
    fig.axes[i].vlines(unit.alleys[alley][i]['spikes'][:,0], -10,10, color='r')
    fig.axes[i].set_title(f"{i}, {txtVisits[alley-1][i]}")
    fig.axes[i].set_xticks([])

In [145]:
phase_totals = {txt:np.empty((0)) for txt in ['A','B','C']}
for trial in range(len(unit.alleys[alley])):
    adjspikes = np.asarray([util.takeClosest(tschunks[trial],i) for i in unit.alleys[alley][trial]['spikes'][:,0]])
    idx = np.searchsorted(tschunks[trial],adjspikes)
    phasevals = phases[trial][idx]
    txt = unit.alleys[alley][trial]['metadata']['stimulus']
    phase_totals[txt] = np.hstack((phase_totals[txt], phasevals))

In [147]:
fig, ax = plt.subplots(1,3, subplot_kw=dict(projection='polar'))
for i, stim in enumerate(['A','B','C']):
    fig.axes[i].hist(phase_totals[stim],bins=18)
    fig.axes[i].set_title(stim)
plt.suptitle(f"{unit.name}, Alley {alley}")

Text(0.5,0.98,'TT6cl-maze1.4, Alley 5')

## Comparing spiking/eeg phase relationship inside/outside putative effect region

In [11]:
targetbins = [3,7]
orientation = "horizontal"
alley = 7

In [12]:
unit = Core.UnitData("TT9\\cl-maze1.6", datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
unit.loadData_raw(includeRewards=True)

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:157: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:157: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:163: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:163: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:321: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


In [13]:
visitTS = alleyVisits[alley-1]
cscchunks = []
tschunks = []

for visit in visitTS:
    mask = (cscts>visit[0])&(cscts<visit[1])
    cscchunks.append(cscdata[mask])
    tschunks.append(cscts[mask])

In [14]:
visits_theta = []
visits_gamma = []

for i,visit in enumerate(alleyVisits[alley-1]):
    theta = filter_data(cscchunks[i], 8,12)
    gamma = filter_data(cscchunks[i], 25, 45)
    visits_theta.append(theta)
    visits_gamma.append(gamma)

In [18]:
oscill = 'gamma'

hilbert3 = lambda x: hilbert(x, fftpack.next_fast_len(len(x)))[:len(x)] # this truncates bc for certain array sizes (primes) underlying FFT is insanely slow.
phases = []
for i in range(len(cscchunks)):
    if oscill == 'gamma':
        analytic = hilbert3(visits_gamma[i])
    elif oscill == 'theta':
        analytic = hilbert3(visits_theta[i])
    phase = np.angle(analytic)
    phases.append(phase)

In [19]:
def checkinside(p, alley, orien, tb):
    if orien == 'horizontal':
        x1, x2 = unit.alleyBins[alley-1]['cols'][tb[0]], unit.alleyBins[alley-1]['cols'][tb[1]+1]
        y1, y2 = unit.alleyBins[alley-1]['rows'][0], unit.alleyBins[alley-1]['rows'][-1]
    elif orien == 'vertical':
        x1, x2 = unit.alleyBins[alley-1]['cols'][0], unit.alleyBins[alley-1]['cols'][-1]
        y1, y2 = unit.alleyBins[alley-1]['rows'][tb[0]], unit.alleyBins[alley-1]['rows'][tb[1]+1]
    
    if p[0] >= x1 and p[0] <= x2 and p[1] >= y1 and p[1] <= y2:
        return True
    else:
        return False

In [20]:
phase_totals_in = {txt:np.empty((0)) for txt in ['A','B','C']}
phase_totals_out = {txt:np.empty((0)) for txt in ['A','B','C']}

for t,visit in enumerate(unit.alleys[alley]):
    if visit['spikes'].shape[0] > 0:
        mask = np.asarray([checkinside(p[1:],alley,orientation,targetbins) for p in visit['spikes']])
        spikesin = visit['spikes'][mask,:]
        spikesout = visit['spikes'][~mask,:]
        txtpresent = visit['metadata']['stimulus']
        
        adjspikes = np.asarray([util.takeClosest(tschunks[t],i) for i in spikesin[:,0]])
        idx = np.searchsorted(tschunks[t],adjspikes)
        phasevals = phases[t][idx]
        phase_totals_in[txtpresent] = np.hstack((phase_totals_in[txtpresent], phasevals))
        
        adjspikes = np.asarray([util.takeClosest(tschunks[t],i) for i in spikesout[:,0]])
        idx = np.searchsorted(tschunks[t],adjspikes)
        phasevals = phases[t][idx]
        phase_totals_out[txtpresent] = np.hstack((phase_totals_out[txtpresent], phasevals))

In [23]:
fig, ax = plt.subplots(2,3, subplot_kw=dict(projection='polar'))

for i, stim in enumerate(['A','B','C']):
    ax[0,i].hist(phase_totals_in[stim],bins=18)
    ax[0,i].set_title(f"{stim}, in window")
    
    ax[1,i].hist(phase_totals_out[stim],bins=18)
    ax[1,i].set_title(f"{stim}, out window")
    
plt.suptitle(f"{unit.name}, Alley {alley}, {oscill} Rhythm")

Text(0.5,0.98,'TT9cl-maze1.6, Alley 7, gamma Rhythm')

## Looking at spiking relation to gamma cycles

In [314]:
trial = 10
periods = []
for i,p in enumerate(phases[trial][:-1]):
    if np.sign(p) == 1 and np.sign(phases[trial][i+1]) == -1:
        periods.append(tschunks[trial][i])

In [342]:
# this is getting gamma cycles (expressed as repeating phases) in/out of target region
txtcolorlookup = {'A':'r', 'B':'b', 'C':'g'}

numrows = 5
fig, ax = plt.subplots(numrows,int(np.ceil(len(alleyVisits[alley-1])/numrows)), figsize=(10,10))

for t,visit in enumerate(unit.alleys[alley]):
    
    periods = []
    for i,p in enumerate(phases[t][:-1]):
        if np.sign(p) == 1 and np.sign(phases[t][i+1]) == -1:
            periods.append(tschunks[t][i])
    
    mask = np.asarray([checkinside(p[1:],alley,orientation,targetbins) for p in visit['occs']])
    occsin = visit['occs'][mask,0]
    begin_region, end_region = occsin[0], occsin[-1]

    txtpresent = visit['metadata']['stimulus']

    fig.axes[t].vlines(periods,-1,1,color='k')
    fig.axes[t].vlines(visit['spikes'][:,0],-0.5,0.5,color=txtcolorlookup[txtpresent])
    fig.axes[t].vlines(begin_region, -2, 2, color='purple')
    fig.axes[t].vlines(end_region, -2, 2, color='purple')
    fig.axes[t].set_xticks([])
    fig.axes[t].set_yticks([])
    fig.axes[t].set_title(t)

## Looking at spike distribution wrt gamma cycles

In [88]:
propCyclesSpikesIn = []
propCyclesSpikesOut = []
hins = []
houts = []
txts = []

for t,visit in enumerate(unit.alleys[alley]):
    if visit['spikes'].shape[0] > 0:
        mask = np.asarray([checkinside(p[1:],alley,orientation,targetbins) for p in visit['spikes']])
        spikesin = visit['spikes'][mask,:]
        spikesout = visit['spikes'][~mask,:]
        txtpresent = visit['metadata']['stimulus']
        txts.append(txtpresent)
        
        periods = []
        for i,p in enumerate(phases[t][:-1]):
            if np.sign(p) == 1 and np.sign(phases[t][i+1]) == -1:
                periods.append(tschunks[t][i])
            
        hin,_ = np.histogram(spikesin[:,0], bins=periods)
        hins.append(hin)
        propsin = np.count_nonzero(hin)/hin.shape[0]
        propCyclesSpikesIn.append(propsin)

        hout,_ = np.histogram(spikesout[:,0], bins=periods)
        houts.append(hout)
        propsout = np.count_nonzero(hout)/hout.shape[0]
        propCyclesSpikesOut.append(propsout)
        